In [423]:
import pandas as pd
import numpy as np
import random
import os
import sys
import requests
import time
import datetime as dt

from dotenv import load_dotenv
from flipside import Flipside
from prophet import Prophet

from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from scripts.utils import flipside_api_results, set_random_seed
# from sql_queries.sql_scripts import three_dns_sales

In [424]:
three_dns_sales = """

SELECT
  DATE_TRUNC('HOUR', BLOCK_TIMESTAMP) AS day, tokenid, tx_hash, price
FROM
  optimism.nft.ez_nft_sales
WHERE
  NFT_ADDRESS = LOWER('0xBB7B805B257d7C76CA9435B3ffe780355E4C4B17')
  AND event_type = 'sale'
"""

In [425]:
current_directory = os.getcwd()
current_directory

'e:\\Projects\\liquid_domains'

In [426]:
load_dotenv()

True

In [427]:
seed = 20
set_random_seed(seed)

In [428]:
flipside_api_key = os.getenv('FLIPSIDE_API_KEY')
alchemy_api_key = os.getenv('ALCHEMY_API_KEY')
opensea_api_key = os.getenv('OPENSEA_API_KEY')

print(alchemy_api_key)

6AUlaGmWe505S7gRPZXVh4YEFgJdYHy5


In [429]:
def alchemy_metadata_api(api_key, network, contract_address):
    if network == 'optimism':
        network = 'opt'
    elif network == 'ethereum':
        network = 'eth'
    elif network == 'base':
        network = 'base'
    # Replace with your actual API key
    base_url = f"https://{network}-mainnet.g.alchemy.com/nft/v3/{api_key}/getNFTsForContract"
    print(f'Base URL: {base_url}')
    headers = {"accept": "application/json"}

    # Pagination parameters
    page_key = None  # Initial key for pagination
    limit = 100  # Set the limit for the number of NFTs per request
    api_data = []  # To store all NFTs

    while True:
        params = {
            "contractAddress": contract_address,
            "withMetadata": "true",
            "limit": limit
        }
        
        if page_key:
            params["pageKey"] = page_key

        response = requests.get(base_url, headers=headers, params=params)
        data = response.json()
        
        if "nfts" in data:
            api_data.extend(data["nfts"])
            # print(data["nfts"])
        
        # Check if there's a next page key for pagination
        page_key = data.get("pageKey", None)
        
        if page_key is None:
            break

        print(f'Number added: {len(data["nfts"])} | Total number of NFTs: {len(api_data)}, Next page key: {page_key}')

    # Now `api_data` contains all the NFTs retrieved from the paginated API calls
    print(f"Total NFTs retrieved: {len(api_data)}")

    # Function to get metadata from tokenUri
    def fetch_metadata(token_uri):
        try:
            response = requests.get(token_uri)
            metadata = response.json()
            return metadata
        except:
            return {'name': 'No name available'}

    # Extract tokenId, name, and tokenUri from each NFT
    nft_info = []
    for nft in api_data:
        token_id = nft.get('tokenId', 'No token ID available')
        token_name = nft.get('name', 'No token ID available')
        
        
        nft_info.append({'tokenId': token_id, 'name': token_name})
    
    # Create DataFrame
    df = pd.DataFrame(nft_info)
    
    return df

optimism_name_service_metadata = alchemy_metadata_api(alchemy_api_key, 'optimism', '0xC16aCAdf99E4540E6f4E6Da816fd6D2A2C6E1d4F')

Three_DNS_metadata = alchemy_metadata_api(alchemy_api_key, 'optimism', '0xBB7B805B257d7C76CA9435B3ffe780355E4C4B17')

optimistic_domains = alchemy_metadata_api(alchemy_api_key, 'optimism', '0xC16aCAdf99E4540E6f4E6Da816fd6D2A2C6E1d4F')

In [430]:
Optimistic_domains_path = 'data/optimistic_domains_metadata.json'
# optimistic_domains.to_json(Optimistic_domains_path, orient='records')
optimistic_domains = pd.read_json(Optimistic_domains_path, orient='records')
# optimistic_domains.drop(columns=['tokenUri'], inplace=True)
optimistic_domains

,tokenId,name
0,0,vitalik.op
1,1,cardenas.op
2,2,davidcardenas.op
3,3,bitcoin.op
4,4,daffy.op
...,...,...
1321,1321,agency.op
1322,1322,trump.op
1323,1323,send.op
1324,1324,mint.op


In [431]:
domain_path = 'data/domain-name-sales.tsv'  
domain_data = pd.read_csv(domain_path, delimiter='\t')

In [432]:
domain_data.set_index('date', inplace=True)
domain_data = domain_data.drop(columns=['venue'])
domain_data.sort_index(inplace=True)
domain_data

,domain,price
date,,
1999-04-01,altavista.com,3250000
1999-04-01,bingo.com,1100000
1999-11-01,fly.com,1500000
1999-12-01,autos.com,2200000
1999-12-01,england.com,2000000
...,...,...
2021-01-01,yes.movie,253
2021-01-01,yopal.com,405
2021-01-01,yougraph.com,1161


In [433]:
def fetch_event_type(api_key, collection, event_type, all_events, params, headers):
    base_url = f"https://api.opensea.io/api/v2/events/collection/{collection}"
    params['event_type'] = event_type
    
    # Load the last timestamp/identifier
    
    page_count = 0
    while True:
        response = requests.get(base_url, headers=headers, params=params)
        if response.status_code == 200:
            data = response.json()
            fetched_events = data.get("asset_events", [])
            all_events.extend(fetched_events)
            
            if fetched_events:
                # Update the last timestamp/identifier to the latest one fetched
                last_event_time = fetched_events[-1].get("created_date")
            
            page_count += 1
            next_cursor = data.get("next")
            print(f"Fetching {event_type}: Page {page_count}, Events Fetched: {len(fetched_events)}, Total Events: {len(all_events)}, next cursor: {next_cursor}")
            
            if next_cursor:
                params['next'] = next_cursor
            else:
                break  # No more pages to fetch

            time.sleep(1)  # Delay between requests
        else:
            print(f"Failed to fetch {event_type} data: HTTP {response.status_code}, Response: {response.text}")
            break

def clean_data(domain_df):
    domain_df['nft_identifier'] = domain_df['nft'].apply(lambda x: x.get('identifier', 'No identifier available') if x else 'No identifier available')
    domain_df['nft_name'] = domain_df['nft'].apply(lambda x: x.get('name', 'No name available') if x else 'No name available')
    domain_df['token_amt_raw'] = domain_df['payment'].apply(lambda x: x.get('quantity', 'No name available') if x else 'No name available')
    domain_df['token_symbol'] = domain_df['payment'].apply(lambda x: x.get('symbol', 'No name available') if x else 'No name available')
    domain_df['token_decimals'] = domain_df['payment'].apply(lambda x: x.get('decimals', 'No name available') if x else 'No name available')
    domain_df['dt'] = pd.to_datetime(domain_df['event_timestamp'], unit='s')

    def wei_to_ether(quantity, decimals):
        try:
            return int(quantity) / (10 ** decimals)
        except ValueError:
            return None

    domain_df['token_amt_clean'] = domain_df.apply(lambda row: wei_to_ether(row['token_amt_raw'], row['token_decimals']) if row['token_amt_raw'] != 'No name available' and row['token_decimals'] != 'No name available' else None, axis=1)
    domain_df.dropna(inplace=True)
    return domain_df

# Display the updated DataFrame




def fetch_all_events(api_key, collection):
    headers = {
        "accept": "application/json",
        "x-api-key": api_key
    }
    params = {
        "limit": 50  # Adjust the limit as needed
    }

    all_events = []

    # Fetch listings
    # fetch_event_type(api_key, collection, "listing", all_events, params.copy(), headers)

    # Fetch sales
    fetch_event_type(api_key, collection, "sale", all_events, params.copy(), headers)

    # Save the fetched events to a DataFrame
    print(f"Total events fetched: {len(all_events)}")
    df = pd.DataFrame(all_events)
    clean_df = clean_data(df)
    return clean_df 





optimism_name_service_data = fetch_all_events(api_key=opensea_api_key,collection='optimism-name-service')


In [434]:
optimism_name_service_path = 'data/optimism_name_service_metadata.json'
# optimism_name_service_data.to_json(optimism_name_service_path, orient='records')
optimism_name_service_data = pd.read_json(optimism_name_service_path, orient='records')
optimism_name_service_data = optimism_name_service_data[['dt','token_symbol','token_amt_clean','nft_identifier','nft_name']]
optimism_name_service_data


,dt,token_symbol,token_amt_clean,nft_identifier,nft_name
0,1717118919000,ETH,0.0007,9.860556e+76,zainal.op
1,1715204651000,ETH,0.0006,5.499786e+75,oksie.op
2,1711976895000,ETH,0.0007,3.975487e+76,ahamad.op
3,1710975355000,ETH,0.0007,5.487830e+75,squanchy.op
4,1710017397000,ETH,0.0007,5.251663e+76,azukii.op
...,...,...,...,...,...
728,1677932343000,ETH,0.0025,1.202193e+76,888666.op
729,1677932205000,ETH,0.0500,2.121290e+76,highstreet.op
730,1677932205000,ETH,0.0500,2.484444e+76,bitget.op
731,1677932205000,ETH,0.0500,5.490150e+76,bybit.op


Three_DNS_data = fetch_all_events(api_key=opensea_api_key,collection='3dns-powered-domains')


In [435]:
three_dns_path = 'data/3dns_metadata.json'
# Three_DNS_data.to_json(three_dns_path, orient='records')
Three_DNS_data = pd.read_json(three_dns_path, orient='records')
# Three_DNS_data.dropna(inplace=True)
Three_DNS_data = Three_DNS_data[['dt','token_symbol','token_amt_clean','nft_identifier','nft_name']]
Three_DNS_data

,dt,token_symbol,token_amt_clean,nft_identifier,nft_name
0,1722706445000,WETH,0.12000,1.036304e+77,03.box
1,1722705143000,ETH,0.00028,6.390746e+76,flon.chain.box
2,1722705143000,ETH,0.00029,7.049451e+76,eua.chain.box
3,1722705143000,ETH,0.00030,1.020766e+77,investecriptos.chain.box
4,1722705143000,ETH,0.00045,1.181224e+76,drivenspyder.chain.box
...,...,...,...,...,...
153,1706970257000,WETH,0.00100,2.609238e+76,696.box
154,1706653465000,WETH,0.11000,5.664799e+76,opensea.box
155,1706203283000,ETH,0.01000,3.647061e+76,08000.xyz
156,1704080589000,ETH,0.15000,9.923409e+76,2024.finance


ens_sales_data = fetch_all_events(api_key=opensea_api_key,collection='ens')


In [436]:
ens_sales_path = 'data/ens_metadata.json'
# ens_sales_data.to_json('data/ens_metadata.json', orient='records', date_format='iso')
ens_data = pd.read_json(ens_sales_path, orient='records')

In [437]:
ens_data = ens_data[['dt','token_symbol','token_amt_clean','nft_identifier','nft_name']]

unstoppable_sales_data = fetch_all_events(api_key=opensea_api_key,collection='unstoppable-domains')


In [438]:
unstoppable_sales_path = 'data/unstoppable_metadata.json'
# unstoppable_sales_data.to_json(unstoppable_sales_path, orient='records', date_format='iso')
unstoppable_sales_data = pd.read_json(unstoppable_sales_path, orient='records')
unstoppable_sales_data = unstoppable_sales_data[['dt','token_symbol','token_amt_clean','nft_identifier','nft_name']]
unstoppable_sales_data

,dt,token_symbol,token_amt_clean,nft_identifier,nft_name
0,2024-07-05T17:01:23.000,ETH,0.00005,7.326097e+76,gaydream.crypto
1,2024-06-17T09:52:47.000,ETH,0.00099,7.992890e+76,officialdaimler.crypto
2,2024-06-17T08:47:35.000,ETH,0.00099,7.201414e+76,officialaudi.crypto
3,2024-06-17T08:35:23.000,ETH,0.00099,5.593531e+76,officialbillgates.crypto
4,2024-06-16T15:07:23.000,ETH,0.00089,9.118995e+76,scaryterry.crypto
...,...,...,...,...,...
3896,2019-12-24T09:10:32.000,WETH,0.10000,8.608991e+76,bittiez.crypto
3897,2019-12-24T09:09:42.000,WETH,0.10000,9.405697e+76,bitties.crypto
3898,2019-12-21T03:55:27.000,WETH,0.04500,4.461632e+76,impeached.crypto
3899,2019-12-19T15:13:15.000,ETH,0.07000,1.761922e+76,ethstaker.crypto


In [439]:
# unstoppable_sales_data['nft_identifier'] = unstoppable_sales_data['nft'].apply(lambda x: x.get('identifier', 'No identifier available') if x else 'No identifier available')
# unstoppable_sales_data['nft_name'] = unstoppable_sales_data['nft'].apply(lambda x: x.get('name', 'No name available') if x else 'No name available')
# unstoppable_sales_data.dropna(inplace=True)
# # Now you can view the DataFrame with the new columns
# print(unstoppable_sales_data[['event_type', 'closing_date', 'nft_identifier', 'nft_name']])
# unstoppable_sales_data = unstoppable_sales_data[['nft_identifier', 'nft_name']]

base_domains_metadata = fetch_all_events(api_key=opensea_api_key,collection='basedomainnames')

In [440]:
base_domains_path = 'data/base_metadata.json'
# base_domains_metadata.to_json(base_domains_path, orient='records')
base_domains_metadata_pd = pd.read_json(base_domains_path, orient='records')
base_domains_metadata_pd = base_domains_metadata_pd[['dt','token_symbol','token_amt_clean','nft_identifier','nft_name']]
base_domains_metadata_pd

,dt,token_symbol,token_amt_clean,nft_identifier,nft_name
0,1719530881000,ETH,0.000010,6625,Vizzycrypto.base
1,1719530857000,ETH,0.000010,6707,Cryptox.base
2,1719112413000,ETH,0.000545,4863,38888.base
3,1719110431000,ETH,0.000545,19245,venice.base
4,1719108841000,ETH,0.000488,9711,13333.base
...,...,...,...,...,...
91,1691618587000,ETH,0.011000,3505,rug.base
92,1691616539000,ETH,0.011000,2704,200.base
93,1691613043000,ETH,0.002500,4695,manga.base
94,1691607621000,ETH,0.003500,3204,bullish.base



# Now you can view the DataFrame with the new columns
print(base_domains_metadata_pd[['dt','event_type', 'closing_date', 'nft_identifier', 'nft_name','token_amt_clean','token_symbol']])
base_domains_metadata_pd = base_domains_metadata_pd[['nft_identifier', 'nft_name','token_amt_clean','token_symbol']]

In [441]:
domain_data

,domain,price
date,,
1999-04-01,altavista.com,3250000
1999-04-01,bingo.com,1100000
1999-11-01,fly.com,1500000
1999-12-01,autos.com,2200000
1999-12-01,england.com,2000000
...,...,...
2021-01-01,yes.movie,253
2021-01-01,yopal.com,405
2021-01-01,yougraph.com,1161


combined_metadata = pd.concat([
    base_domains_metadata_pd.dropna(),
    unstoppable_sales_data.dropna(),
    ens_data.dropna(),
    Optimistic_domains_metadata_pd.dropna(),
    Three_DNS_metadata_pd.dropna(),
    optimism_name_service_metadata_pd.dropna()
], ignore_index=True)

combined_metadata.rename(columns={"tokenId":"TOKENID"}, inplace=True)

combined_metadata['TOKENID'].describe()

# Sales

In [442]:
ens_sales = pd.read_csv('data/ens_domain_sales.csv')
optimistic_domains_sales = pd.read_csv('data/optimistic_domains_sales.csv')
optimism_domain_service_sales = pd.read_csv('data/optimism_name_service_sales.csv')
base_domains_sales = pd.read_csv('data/base_domain_names_sales.csv')
unstoppable_domains_sales = pd.read_csv('data/unstoppable_domains_sales.csv')
three_dns_sales_data = pd.read_csv('data/three_dns_sales.csv')
prices_data = pd.read_csv('data/prices.csv')

In [443]:
prices_data = prices_data.dropna()
prices_data['SYMBOL'] = prices_data['SYMBOL'].replace('WETH', 'ETH')


prices_data = prices_data.pivot(index='DT',columns='SYMBOL',values='PRICE')
prices_data = prices_data.reset_index()
prices_data

SYMBOL,DT,ETH,MATIC
0,2018-06-16 05:00:00.000,489.493,NaN
1,2018-06-16 06:00:00.000,490.076,NaN
2,2018-06-16 07:00:00.000,492.068,NaN
3,2018-06-16 08:00:00.000,494.070,NaN
4,2018-06-16 09:00:00.000,491.775,NaN
...,...,...,...
53769,2024-08-03 14:00:00.000,3004.780,0.470591
53770,2024-08-03 15:00:00.000,3003.120,0.469327
53771,2024-08-03 16:00:00.000,3004.180,0.468697
53772,2024-08-03 17:00:00.000,2952.170,0.461290


In [444]:
combined_sales = pd.concat([
    ens_sales.dropna(),
    optimistic_domains_sales.dropna(),
    optimism_domain_service_sales.dropna(),
    base_domains_sales.dropna(),
    unstoppable_domains_sales.dropna(),
    three_dns_sales_data.dropna()
], ignore_index=True)

In [445]:
combined_sales = combined_sales.drop_duplicates()
combined_sales['DAY'] = pd.to_datetime(combined_sales['DAY'], errors='coerce')
combined_sales = combined_sales.sort_values(by='DAY')
combined_sales = combined_sales.reset_index(drop=True)
combined_sales


,DAY,TOKENID,PRICE,PRICE_USD
0,2021-12-08 10:00:00,6.326923e+76,5.000000,11.905779
1,2022-02-14 12:00:00,8.866164e+75,5.000000,8.069360
2,2022-02-17 07:00:00,1.250476e+76,2.500000,4.517472
3,2022-02-23 00:00:00,2.208510e+76,3.000000,4.318938
4,2022-04-12 19:00:00,1.203733e+76,1.990000,2.801802
...,...,...,...,...
106869,2024-08-02 20:00:00,1.127672e+77,0.000450,1.357668
106870,2024-08-03 03:00:00,3.017468e+76,0.100000,294.934000
106871,2024-08-03 07:00:00,8.531689e+76,0.000280,0.830497
106872,2024-08-03 08:00:00,1.016763e+77,0.010000,29.843200


# Full Data Set and Feature Engineering

In [446]:
optimistic_domains_sales

,DAY,TOKENID,PRICE,PRICE_USD
0,2022-07-28 16:00:00.000,850.0,0.00500,8.270859
1,2022-07-13 06:00:00.000,902.0,0.02000,21.050455
2,2022-07-13 06:00:00.000,915.0,0.02500,26.313069
3,2022-08-04 20:00:00.000,933.0,0.01420,22.764884
4,2022-08-04 20:00:00.000,932.0,0.01337,21.434260
5,2022-06-19 00:00:00.000,787.0,0.02000,19.680912
6,2022-10-28 09:00:00.000,1069.0,0.02000,29.977379
7,2022-12-29 07:00:00.000,759.0,0.01100,13.147811
8,2022-06-03 04:00:00.000,482.0,0.01500,27.465282
9,2022-06-04 02:00:00.000,550.0,0.01500,26.637720


In [447]:
optimistic_domains_sales = optimistic_domains_sales.dropna(subset=['TOKENID'])
optimistic_domains_sales['TOKENID']

0      850.0
1      902.0
2      915.0
3      933.0
4      932.0
5      787.0
6     1069.0
7      759.0
8      482.0
9      550.0
10     549.0
11     347.0
12     731.0
13     413.0
14    1076.0
15     576.0
16     413.0
Name: TOKENID, dtype: float64

In [448]:
optimistic_domains_sales['TOKENID'] = optimistic_domains_sales['TOKENID'].astype(int)
optimistic_domains_sales.rename(columns={"TOKENID":"tokenId"}, inplace=True)

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_83436\167122248.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  optimistic_domains_sales['TOKENID'] = optimistic_domains_sales['TOKENID'].astype(int)
C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_83436\167122248.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  optimistic_domains_sales.rename(columns={"TOKENID":"tokenId"}, inplace=True)


In [449]:
optimistic_domains_sales['tokenId']

0      850
1      902
2      915
3      933
4      932
5      787
6     1069
7      759
8      482
9      550
10     549
11     347
12     731
13     413
14    1076
15     576
16     413
Name: tokenId, dtype: int64

In [450]:
optimistic_domains['tokenId']


0          0
1          1
2          2
3          3
4          4
        ... 
1321    1321
1322    1322
1323    1323
1324    1324
1325    1325
Name: tokenId, Length: 1326, dtype: int64

In [451]:
optimistic_data = pd.merge(optimistic_domains_sales, optimistic_domains, on='tokenId', how='left')
optimistic_data.rename(columns={"tokenId":"nft_identifier","name":"nft_name", "day":"dt"}, inplace=True)

In [452]:
prices_data

SYMBOL,DT,ETH,MATIC
0,2018-06-16 05:00:00.000,489.493,NaN
1,2018-06-16 06:00:00.000,490.076,NaN
2,2018-06-16 07:00:00.000,492.068,NaN
3,2018-06-16 08:00:00.000,494.070,NaN
4,2018-06-16 09:00:00.000,491.775,NaN
...,...,...,...
53769,2024-08-03 14:00:00.000,3004.780,0.470591
53770,2024-08-03 15:00:00.000,3003.120,0.469327
53771,2024-08-03 16:00:00.000,3004.180,0.468697
53772,2024-08-03 17:00:00.000,2952.170,0.461290


In [453]:
optimism_name_service_data['dt'] = pd.to_datetime(optimism_name_service_data['dt'], unit='ms')
Three_DNS_data['dt'] = pd.to_datetime(Three_DNS_data['dt'], unit='ms')
ens_data['dt'] = pd.to_datetime(ens_data['dt'])
unstoppable_sales_data['dt'] = pd.to_datetime(unstoppable_sales_data['dt'])
base_domains_metadata_pd['dt'] = pd.to_datetime(base_domains_metadata_pd['dt'], unit='ms')


optimism_name_service_data

,dt,token_symbol,token_amt_clean,nft_identifier,nft_name
0,2024-05-31 01:28:39,ETH,0.0007,9.860556e+76,zainal.op
1,2024-05-08 21:44:11,ETH,0.0006,5.499786e+75,oksie.op
2,2024-04-01 13:08:15,ETH,0.0007,3.975487e+76,ahamad.op
3,2024-03-20 22:55:55,ETH,0.0007,5.487830e+75,squanchy.op
4,2024-03-09 20:49:57,ETH,0.0007,5.251663e+76,azukii.op
...,...,...,...,...,...
728,2023-03-04 12:19:03,ETH,0.0025,1.202193e+76,888666.op
729,2023-03-04 12:16:45,ETH,0.0500,2.121290e+76,highstreet.op
730,2023-03-04 12:16:45,ETH,0.0500,2.484444e+76,bitget.op
731,2023-03-04 12:16:45,ETH,0.0500,5.490150e+76,bybit.op


In [454]:
def hourly(df):
    df['dt'] = df['dt'].dt.strftime('%Y-%m-%d %H-00-00')
    df['dt'] = pd.to_datetime(df['dt'])
    return df


In [455]:
Three_DNS_data = hourly(Three_DNS_data)
optimism_name_service_data = hourly(optimism_name_service_data)
ens_data = hourly(ens_data)
unstoppable_sales_data = hourly(unstoppable_sales_data)
base_domains_metadata_pd = hourly(base_domains_metadata_pd)

Three_DNS_data

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_83436\36066411.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['dt'] = pd.to_datetime(df['dt'])
C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_83436\36066411.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['dt'] = pd.to_datetime(df['dt'])
C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_83436\36066411.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['dt'] = pd.to_datetime(df['dt'])
C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_83436\36066411.py:3: UserWarning: Could not infer format, so each eleme

,dt,token_symbol,token_amt_clean,nft_identifier,nft_name
0,2024-08-03 17:00:00+00:00,WETH,0.12000,1.036304e+77,03.box
1,2024-08-03 17:00:00+00:00,ETH,0.00028,6.390746e+76,flon.chain.box
2,2024-08-03 17:00:00+00:00,ETH,0.00029,7.049451e+76,eua.chain.box
3,2024-08-03 17:00:00+00:00,ETH,0.00030,1.020766e+77,investecriptos.chain.box
4,2024-08-03 17:00:00+00:00,ETH,0.00045,1.181224e+76,drivenspyder.chain.box
...,...,...,...,...,...
153,2024-02-03 14:00:00+00:00,WETH,0.00100,2.609238e+76,696.box
154,2024-01-30 22:00:00+00:00,WETH,0.11000,5.664799e+76,opensea.box
155,2024-01-25 17:00:00+00:00,ETH,0.01000,3.647061e+76,08000.xyz
156,2024-01-01 03:00:00+00:00,ETH,0.15000,9.923409e+76,2024.finance


In [456]:
Three_DNS_data['dt']

0     2024-08-03 17:00:00+00:00
1     2024-08-03 17:00:00+00:00
2     2024-08-03 17:00:00+00:00
3     2024-08-03 17:00:00+00:00
4     2024-08-03 17:00:00+00:00
                 ...           
153   2024-02-03 14:00:00+00:00
154   2024-01-30 22:00:00+00:00
155   2024-01-25 17:00:00+00:00
156   2024-01-01 03:00:00+00:00
157   2023-12-26 21:00:00+00:00
Name: dt, Length: 158, dtype: datetime64[ns, UTC]

In [457]:
prices_data['DT'] = pd.to_datetime(prices_data['DT'])
prices_data.rename(columns={'DT':'dt'}, inplace=True)


In [458]:
prices_data['dt'] = prices_data['dt'].dt.tz_localize('UTC')
prices_data

SYMBOL,dt,ETH,MATIC
0,2018-06-16 05:00:00+00:00,489.493,NaN
1,2018-06-16 06:00:00+00:00,490.076,NaN
2,2018-06-16 07:00:00+00:00,492.068,NaN
3,2018-06-16 08:00:00+00:00,494.070,NaN
4,2018-06-16 09:00:00+00:00,491.775,NaN
...,...,...,...
53769,2024-08-03 14:00:00+00:00,3004.780,0.470591
53770,2024-08-03 15:00:00+00:00,3003.120,0.469327
53771,2024-08-03 16:00:00+00:00,3004.180,0.468697
53772,2024-08-03 17:00:00+00:00,2952.170,0.461290


In [459]:
Three_DNS_data = Three_DNS_data.merge(prices_data, how='left', on='dt')
Three_DNS_data['price_usd'] = Three_DNS_data['token_amt_clean'] * Three_DNS_data['ETH']
Three_DNS_data

,dt,token_symbol,token_amt_clean,nft_identifier,nft_name,ETH,MATIC,price_usd
0,2024-08-03 17:00:00+00:00,WETH,0.12000,1.036304e+77,03.box,2952.170000,0.461290,354.260400
1,2024-08-03 17:00:00+00:00,ETH,0.00028,6.390746e+76,flon.chain.box,2952.170000,0.461290,0.826608
2,2024-08-03 17:00:00+00:00,ETH,0.00029,7.049451e+76,eua.chain.box,2952.170000,0.461290,0.856129
3,2024-08-03 17:00:00+00:00,ETH,0.00030,1.020766e+77,investecriptos.chain.box,2952.170000,0.461290,0.885651
4,2024-08-03 17:00:00+00:00,ETH,0.00045,1.181224e+76,drivenspyder.chain.box,2952.170000,0.461290,1.328477
...,...,...,...,...,...,...,...,...
153,2024-02-03 14:00:00+00:00,WETH,0.00100,2.609238e+76,696.box,2306.157346,0.789116,2.306157
154,2024-01-30 22:00:00+00:00,WETH,0.11000,5.664799e+76,opensea.box,2372.074784,0.814667,260.928226
155,2024-01-25 17:00:00+00:00,ETH,0.01000,3.647061e+76,08000.xyz,2187.105106,0.718289,21.871051
156,2024-01-01 03:00:00+00:00,ETH,0.15000,9.923409e+76,2024.finance,2298.241468,0.989499,344.736220


In [460]:
optimism_name_service_data = optimism_name_service_data.merge(prices_data, how='left', on='dt')
optimism_name_service_data['price_usd'] = optimism_name_service_data['token_amt_clean'] * optimism_name_service_data['ETH']
optimism_name_service_data


,dt,token_symbol,token_amt_clean,nft_identifier,nft_name,ETH,MATIC,price_usd
0,2024-05-31 01:00:00+00:00,ETH,0.0007,9.860556e+76,zainal.op,3747.700000,0.696484,2.623390
1,2024-05-08 21:00:00+00:00,ETH,0.0006,5.499786e+75,oksie.op,2996.856179,0.686463,1.798114
2,2024-04-01 13:00:00+00:00,ETH,0.0007,3.975487e+76,ahamad.op,3540.430803,0.960743,2.478302
3,2024-03-20 22:00:00+00:00,ETH,0.0007,5.487830e+75,squanchy.op,3465.299102,1.008320,2.425709
4,2024-03-09 20:00:00+00:00,ETH,0.0007,5.251663e+76,azukii.op,3894.488835,1.132292,2.726142
...,...,...,...,...,...,...,...,...
728,2023-03-04 12:00:00+00:00,ETH,0.0025,1.202193e+76,888666.op,1570.376347,1.149929,3.925941
729,2023-03-04 12:00:00+00:00,ETH,0.0500,2.121290e+76,highstreet.op,1570.376347,1.149929,78.518817
730,2023-03-04 12:00:00+00:00,ETH,0.0500,2.484444e+76,bitget.op,1570.376347,1.149929,78.518817
731,2023-03-04 12:00:00+00:00,ETH,0.0500,5.490150e+76,bybit.op,1570.376347,1.149929,78.518817


In [461]:
ens_data = ens_data.merge(prices_data, how='left', on='dt')
ens_data['price_usd'] = ens_data['token_amt_clean'] * ens_data['ETH']
ens_data

,dt,token_symbol,token_amt_clean,nft_identifier,nft_name,ETH,MATIC,price_usd
0,2024-08-03 15:00:00+00:00,ETH,0.002455,8.634910e+76,ghaut.eth,3003.120000,0.469327,7.372660
1,2024-08-03 14:00:00+00:00,ETH,0.007000,7.155201e+76,ltaliano.eth,3004.780000,0.470591,21.033460
2,2024-08-03 13:00:00+00:00,ETH,0.002000,3.691040e+76,9797th.eth,2995.200000,0.471277,5.990400
3,2024-08-03 09:00:00+00:00,WETH,0.550000,8.066579e+76,joule.eth,2982.290000,0.471256,1640.259500
4,2024-08-03 08:00:00+00:00,ETH,0.010000,1.016763e+77,domainbank.eth,2984.320000,0.471124,29.843200
...,...,...,...,...,...,...,...,...
41151,2023-06-12 05:00:00+00:00,ETH,0.009500,2.643352e+76,82337.eth,1735.287583,0.622909,16.485232
41152,2023-06-12 05:00:00+00:00,ETH,0.009500,6.924902e+76,60983.eth,1735.287583,0.622909,16.485232
41153,2023-06-12 05:00:00+00:00,ETH,0.009500,9.059089e+76,22967.eth,1735.287583,0.622909,16.485232
41154,2023-06-12 05:00:00+00:00,ETH,0.009500,9.653130e+76,55287.eth,1735.287583,0.622909,16.485232


In [462]:
unstoppable_sales_data = unstoppable_sales_data.merge(prices_data, how='left', on='dt')
unstoppable_sales_data['price_usd'] = unstoppable_sales_data['token_amt_clean'] * unstoppable_sales_data['ETH']
unstoppable_sales_data


,dt,token_symbol,token_amt_clean,nft_identifier,nft_name,ETH,MATIC,price_usd
0,2024-07-05 17:00:00+00:00,ETH,0.00005,7.326097e+76,gaydream.crypto,2985.490000,0.462827,0.149275
1,2024-06-17 09:00:00+00:00,ETH,0.00099,7.992890e+76,officialdaimler.crypto,3553.730000,0.605677,3.518193
2,2024-06-17 08:00:00+00:00,ETH,0.00099,7.201414e+76,officialaudi.crypto,3560.490000,0.605170,3.524885
3,2024-06-17 08:00:00+00:00,ETH,0.00099,5.593531e+76,officialbillgates.crypto,3560.490000,0.605170,3.524885
4,2024-06-16 15:00:00+00:00,ETH,0.00089,9.118995e+76,scaryterry.crypto,3579.490000,0.619505,3.185746
...,...,...,...,...,...,...,...,...
3896,2019-12-24 09:00:00+00:00,WETH,0.10000,8.608991e+76,bittiez.crypto,127.239335,0.014973,12.723934
3897,2019-12-24 09:00:00+00:00,WETH,0.10000,9.405697e+76,bitties.crypto,127.239335,0.014973,12.723934
3898,2019-12-21 03:00:00+00:00,WETH,0.04500,4.461632e+76,impeached.crypto,127.466282,0.015561,5.735983
3899,2019-12-19 15:00:00+00:00,ETH,0.07000,1.761922e+76,ethstaker.crypto,126.455708,0.012581,8.851900


In [463]:
base_domains_metadata_pd = base_domains_metadata_pd.merge(prices_data, how='left', on='dt')
base_domains_metadata_pd['price_usd'] = base_domains_metadata_pd['token_amt_clean'] * base_domains_metadata_pd['ETH']
base_domains_metadata_pd


,dt,token_symbol,token_amt_clean,nft_identifier,nft_name,ETH,MATIC,price_usd
0,2024-06-27 23:00:00+00:00,ETH,0.000010,6625,Vizzycrypto.base,3443.160000,0.569368,0.034432
1,2024-06-27 23:00:00+00:00,ETH,0.000010,6707,Cryptox.base,3443.160000,0.569368,0.034432
2,2024-06-23 03:00:00+00:00,ETH,0.000545,4863,38888.base,3505.750000,0.575821,1.910283
3,2024-06-23 02:00:00+00:00,ETH,0.000545,19245,venice.base,3506.130000,0.573719,1.910490
4,2024-06-23 02:00:00+00:00,ETH,0.000488,9711,13333.base,3506.130000,0.573719,1.710991
...,...,...,...,...,...,...,...,...
91,2023-08-09 22:00:00+00:00,ETH,0.011000,3505,rug.base,1851.525326,0.685655,20.366779
92,2023-08-09 21:00:00+00:00,ETH,0.011000,2704,200.base,1851.516479,0.684431,20.366681
93,2023-08-09 20:00:00+00:00,ETH,0.002500,4695,manga.base,1852.535651,0.685387,4.631339
94,2023-08-09 19:00:00+00:00,ETH,0.003500,3204,bullish.base,1847.666102,0.684107,6.466831


In [466]:
optimistic_data.rename(columns={'DAY':'dt','PRICE_USD':'price_usd','PRICE':'token_amt_clean'}, inplace=True)

In [475]:
optimistic_data['dt'] = pd.to_datetime(optimistic_data['dt'])
optimistic_data['dt'] = optimistic_data['dt'].dt.tz_localize('UTC')
optimistic_data['dt'] = pd.to_datetime(optimistic_data['dt'])


In [469]:
base_domains_metadata_pd

,dt,token_symbol,token_amt_clean,nft_identifier,nft_name,ETH,MATIC,price_usd
0,2024-06-27 23:00:00+00:00,ETH,0.000010,6625,Vizzycrypto.base,3443.160000,0.569368,0.034432
1,2024-06-27 23:00:00+00:00,ETH,0.000010,6707,Cryptox.base,3443.160000,0.569368,0.034432
2,2024-06-23 03:00:00+00:00,ETH,0.000545,4863,38888.base,3505.750000,0.575821,1.910283
3,2024-06-23 02:00:00+00:00,ETH,0.000545,19245,venice.base,3506.130000,0.573719,1.910490
4,2024-06-23 02:00:00+00:00,ETH,0.000488,9711,13333.base,3506.130000,0.573719,1.710991
...,...,...,...,...,...,...,...,...
91,2023-08-09 22:00:00+00:00,ETH,0.011000,3505,rug.base,1851.525326,0.685655,20.366779
92,2023-08-09 21:00:00+00:00,ETH,0.011000,2704,200.base,1851.516479,0.684431,20.366681
93,2023-08-09 20:00:00+00:00,ETH,0.002500,4695,manga.base,1852.535651,0.685387,4.631339
94,2023-08-09 19:00:00+00:00,ETH,0.003500,3204,bullish.base,1847.666102,0.684107,6.466831


In [476]:
combined_dataset = pd.concat([
    ens_data[['dt','nft_name','price_usd','token_amt_clean']].dropna(),
    optimistic_data[['dt','nft_name','price_usd','token_amt_clean']].dropna(),
    optimism_name_service_data[['dt','nft_name','price_usd','token_amt_clean']].dropna(),
    unstoppable_sales_data[['dt','nft_name','price_usd','token_amt_clean']].dropna(),
    base_domains_metadata_pd[['dt','nft_name','price_usd','token_amt_clean']].dropna(),
    Three_DNS_data[['dt','nft_name','price_usd','token_amt_clean']].dropna()
], ignore_index=True)

combined_dataset = combined_dataset.drop_duplicates()
combined_dataset['dt'] = pd.to_datetime(combined_dataset['dt'], errors='coerce')
combined_dataset = combined_dataset.sort_values(by='dt')
combined_dataset = combined_dataset.reset_index(drop=True)
combined_dataset


,dt,nft_name,price_usd,token_amt_clean
0,2019-12-14 08:00:00+00:00,cryptoq.crypto,10.052900,0.06990
1,2019-12-19 15:00:00+00:00,ethstaker.crypto,8.851900,0.07000
2,2019-12-21 03:00:00+00:00,impeached.crypto,5.735983,0.04500
3,2019-12-24 09:00:00+00:00,bitties.crypto,12.723934,0.10000
4,2019-12-24 09:00:00+00:00,bittiez.crypto,12.723934,0.10000
...,...,...,...,...
45911,2024-08-03 17:00:00+00:00,03.box,354.260400,0.12000
45912,2024-08-03 17:00:00+00:00,flon.chain.box,0.826608,0.00028
45913,2024-08-03 17:00:00+00:00,eua.chain.box,0.856129,0.00029
45914,2024-08-03 17:00:00+00:00,investecriptos.chain.box,0.885651,0.00030


In [478]:
combined_dataset.describe()

,price_usd,token_amt_clean
count,4.591600e+04,45916.000000
mean,1.279554e+04,4.626290
std,1.187186e+06,360.718556
min,0.000000e+00,0.000000
25%,2.400513e+00,0.001000
50%,6.077052e+00,0.002525
75%,5.518269e+01,0.029900
max,2.395873e+08,69000.000000
